# Confirming the violation of the CHSH inequality

In this first exercise, we will confirm that the device that we call a quantum computer indeed exhibits quantum mechanical behavior &mdash; entanglement, in particular. You will be introduced to the concepts of quantum mechanics and the fundamentals of quantum computing through this exercise.

```{contents} Contents
---
local: true
---
```

$\newcommand{\ket}[1]{|#1\rangle}$

## Is this really a quantum computer?

The aim of this workbook is to familiarize yourself with quantum computers (QCs), but until just a few years ago, QCs were something that only existed in science fiction. However, now we are told that they are available as real computational resources over the cloud &mdash; but are these devices really QCs? How can we check?

The fundamental way that QCs work is that they manipulate a physical system, composed of elements such as superconducting resonators or cold atoms, so that the results of computations are expressed in the quantum state of the system. In other words, a device can be called a quantum computer only when it contains a physical system whose quantum state can be manipulated in certain ways and also be retained for a long period of time. There must also be algorithms that translate abstract calculations into tangible operations on the physical system. The algorithm portion will be introduced little by little throughout this workbook. Here, let's confirm that our "QC" really operates a quantum mechanical system.

## The CHSH inequality

One way to experimentally verify that a system behaves quantum mechanically is to test the violation of the CHSH inequality{cite}`PhysRevLett.23.880,nielsen_chuang_epr`. To put it briefly, the CHSH inequality is an inequality of specific observables in a two-body system that is satisfied unless there are quantum-specific phenomena such as entanglement. Said more straightforwardly, if you measure these observables in a device and their values violate the CHSH inequality, the device may actually be using quantum phenomena.

Normally, this type of experiment would require a highly sophisticated lab setup (involving laser, non-linear crystals, cold atoms, etc.). But with a cloud-based QC, all that's needed is a web browser. In this workbook, you will use Jupyter Notebook to write Python programs and then use [IBM Q System One](https://www.ibm.com/quantum-computing/systems/) machines through [IBM Quantum](https://quantum-computing.ibm.com/).

## The basic structure of Qiskit

[Qiskit](https://qiskit.org/), a Python library provided by IBM, is used for programming IBM quantum computers. The basic procedure for using Qiskit is as follows:

1. Decide on the number of quantum bits to use.
1. Apply quantum computation operations (gates) to the quantum bits to create a quantum circuit.
1. Execute the circuit and produce calculation results. Here, there are two options:
   - Send the circuit to the actual QC device and get the results back.
   - Simulate the circuit on your computer.
1. Analyze the calculation results.

You will perform this process below, while being introduced to various important concepts. In this exercise, you will only use the actual device. Please refer to the first assignment for information on simulating circuits.

# Quantum bits and quantum registers

**Quantum bits**, or **qubits**, are the fundamental elements that make up the quantum computer. They are the smallest possible unit of quantum information. A number of quantum bits gathered together is referred to as a quantum register.

A quantum register in a quantum computer is always in one "state." Following a common practice used in physics, the states of quantum registers are referred to as "kets" and denoted with a symbol like $\ket{\psi}$[^mixed_state]. If you are unfamiliar with quantum mechanics, this notation method may look intimidating, but the ket itself is merely a symbol, so there is no need to be overly concerned. You could also write it as $\psi$, without enclosing, or even use the 🔱 emoji if you want. Anything works!

What's important is that two **basis states** can be defined for each qubit. Conventionally, the two basis states are labled as $\ket{0}$ and $\ket{1}$, and form what is called a computational basis. Any state of a qubit can then be expressed through a *superposition* of the two computational basis states, using complex numbers $\alpha$ and $\beta$:

$$
\alpha \ket{0} + \beta \ket{1}
$$

In quantum mechanics, the coefficients $\alpha$ and $\beta$ are called the probability amplitudes, or simply **amplitudes**. Again, the actual formatting used is not particularly significant. The states could just as well be written as $[\alpha, \beta]$[^complexarray].

Another way to look at this is that a single qubit carries an amount of information equivalent to two complex numbers. However, there is a caveat: Due to the rules of quantum mechanics, the amplitudes $\alpha$ and $\beta$ must satisfy the following requirement.

$$
|\alpha|^2 + |\beta|^2 = 1
$$

Furthermore, the overall phase of a quantum state is not physically meaningful. In other words,

$$
\alpha \ket{0} + \beta \ket{1} \sim e^{i\theta} (\alpha \ket{0} + \beta \ket{1}),
$$

where $\sim$ indicates that the two sides represents the same quantum state.

A single complex number can be written using two real numbers, so $\alpha$ and $\beta$ together would appear to have the same amount of information as four real numbers, but due to these two constraints, the actual degree of freedom is 4-2=2. Another expression of the state of a qubit that makes the number of degrees of freedom more explicit is

$$
e^{-i\phi/2}\cos\frac{\theta}{2}\ket{0} + e^{i\phi/2}\sin\frac{\theta}{2}\ket{1},
$$

which is sometimes called the Bloch sphere notation.

Things get more interesting when there are multiple qubits. For example, if there are two, each has a $\ket{0}, \ket{1}$ computational basis, so the overall state is a superposition parametrized by four complex numbers,

$$
\alpha \ket{0}\ket{0} + \beta \ket{0}\ket{1} + \gamma \ket{1}\ket{0} + \delta \ket{1}\ket{1}.
$$

The "products" of the computational basis states of the two qubits, $\ket{0}\ket{0}, \ket{0}\ket{1}, \ket{1}\ket{0}$, and $\ket{1}\ket{1}$, are the computational basis states of this quantum register. Their abbreviated notations are $\ket{00}, \ket{01}, \ket{10}$, and $\ket{11}$.

The rules of quantum mechanics regarding the amplitudes in this case are

$$
|\alpha|^2 + |\beta|^2 + |\gamma|^2 + |\delta|^2 = 1
$$

and

$$
\alpha \ket{00} + \beta \ket{01} + \gamma \ket{10} + \delta \ket{11} \sim e^{i\theta} (\alpha \ket{00} + \beta \ket{01} + \gamma \ket{10} + \delta \ket{11}).
$$

There are only two constraints, regardless of the number of qubits.

In other words, a register with $n$ qubits has $2^n$ basis states with a complex amplitude for each, so *the amount of information that it carries is equivalent to $2 \times 2^n - 2$ real numbers*. This is why the word "exponential" often appears in the discussion of quantum calculation.

There is another frequently used notation for the computational basis states of quantum registers. One could look at the string of 0/1s that appears in the ket as a binary number and express it with the corresponding decimal number. For example, the four-qubit register states $\ket{0000}$ and $\ket{1111}$ can be expressed as $\ket{0}$ and $\ket{15}$, respectively. However, in this case, care must be taken to clearly indicate which qubit, the leftmost or the rightmost, corresponds to the least significant bit (LSB). Whether $\ket{0100}$ becomes $\ket{4}$ (the rightmost qubit is the LSB) or $\ket{2}$ (the leftmost is the LSB) depends on which convention is used. In this workbook, in accordance with the definition used in Qiskit, we take the rightmost qubit to be the LSB. Furthermore, we want to make the first qubit of the register correspond to the LSB. Therefore, when expressing a computational basis state with an array of kets or 0/1s, the register's qubits will be arranged from right to left.

Qiskit has a quantum register class, whose instance is created as:
```{code-block} python
from qiskit import QuantumRegister
register = QuantumRegister(4, 'myregister')
```
i.e., by specifying the number of qubits (four in this case) and the name (`'myregister'`). By default, all qubits will be in the |0⟩ state. The register object is not very useful by itself. Instead, they are usually used as parts in quantum circuits, which are introduced below.

[^mixed_state]: Strictly speaking, the state of a register can be expressed with kets only when the register is not entangled with other registers, but we will skip the details here.
[^complexarray]: In the simulations of QCs on classical computers, the states of the quantum registers are expressed with arrays of complex numbers, which corresponds well with this notation.

### ゲート、回路、測定

量子計算とは、端的に言えば、量子レジスタに特定の状態を生成し、その振幅を利用することと言えます。

とは言っても、いきなり「えいや」と好きな量子状態を作れるわけではなく、パターンの決まった単純操作（$\ket{0}$と$\ket{1}$を入れ替える、ブロッホ球表現での位相角度$\phi$を増減させる、など）を順番に組み合わせて複雑な状態を作っていきます。この単純操作のオペレーションのことを一般に量子**ゲート**といい、ゲートの種類や順番を指定したプログラムに相当するものを量子**回路**と呼びます。

Qiskitでは、量子回路を`QuantumCircuit`オブジェクトで表します。
```{code-block} python
from qiskit import QuantumCircuit, QuantumRegister
register = QuantumRegister(4, 'myregister')
circuit = QuantumCircuit(register)
```
という具合です。

作られた量子回路は、量子ビットの数が決まっているもののゲートが一つもない「空っぽ」の状態なので、そこにゲートをかけていきます。例えば下で説明するアダマールゲートをレジスタの2個目の量子ビットに作用させるには
```{code-block} python
circuit.h(register[1])
```
とします。

上で「振幅を利用する」という曖昧な表現をしましたが、それはいろいろな利用の仕方があるからです。しかし、どんな方法であっても、必ず量子レジスタの**測定**という操作を行います。量子コンピュータから何かしらの情報を得るための唯一の方法が測定です。
 
測定は量子レジスタの状態を「覗き見る」ような操作ですが、一回の測定操作で具体的に起きることは、各量子ビットに対して0もしくは1という値が得られるというだけです。つまり、量子状態が$2^n$個の計算基底の複雑な重ね合わせであったとしても、測定をすると一つの計算基底に対応するビット列が出てくるだけということになります。しかも、一度測定してしまった量子ビットはもう状態を変えてしまっていて、複雑な重ね合わせは失われてしまいます。

ではこの「一つの計算基底」がどの基底なのかというと、実は特殊な場合を除いて決まっていません。全く同じ回路を何度も実行して測定を繰り返すと、その都度ランダムにビット列が決まります。ただし、このランダムさには法則があって、**特定のビット列が得られる確率は、対応する計算基底の振幅の絶対値自乗**となっています。つまり、$n$ビットレジスタの状態$\sum_{j=0}^{2^n-1} c_j \ket{j}$があるとき、測定でビット列$k$が得られる確率は$|c_k|^2$です。
 
したがって、回路の実行と測定を何度も繰り返して、それぞれのビット列が現れる頻度を記録すれば、だんだん$|c_j|^2$の値がわかっていき、レジスタがどんな量子状態にあるのか見えてきます。しかし、わかるのは$c_j$の絶対値だけで、複素位相については知る術なしです。どうもすっきりしませんが、これが量子コンピュータから情報を得る方法です。

逆に、指数関数的な内部の情報量をうまく使って計算を行いつつ、測定という限定的な方法でも答えが読み出せるように工夫するのが、量子アルゴリズム設計の真髄ということになります。例えば理想的には、何か計算の答えが整数$k$であり、それを計算する回路の終状態が単純に$\ket{k}$となるようであれば、一度の測定で答えがわかる（上でいった特殊な場合に相当）わけです。単純に$\ket{k}$でなくても、重ね合わせ$\sum_{j=0}^{2^n-1} c_j \ket{j}$において$|c_k| \gg |c_{j \neq k}|$を実現できれば、数回の測定で答えが高確率でわかります。{doc}`shor`で紹介する位相推定アルゴリズムはその好例です。

### よく使うゲート

IBM Q System Oneのような超電導振動子を利用した量子コンピュータでは、実際に使用できるゲートは量子ビット1つにかかるものと2つにかかるものに限定されます。しかし、それらを十分な数組み合わせれば、$n$量子ビットレジスタにおいてどのような状態も実現できることが、数学的に証明できます。

#### 1量子ビットの操作

1量子ビットの操作でよく使われるゲートには、以下のようなものがあります。（表中コードの`i`, `j`は量子ビットの番号）

```{list-table}
:header-rows: 1
* - ゲート名
  - 説明
  - Qiskitコード
* - $X$
  - $\ket{0}$と$\ket{1}$を入れ替える。
  - `circuit.x(i)`
* - $Z$
  - $\ket{1}$の振幅を$-1$倍する。
  - `circuit.z(i)`
* - $H$（アダマールゲート）
  - 計算基底それぞれに対して、以下の変形をする。
    ```{math}
    H\ket{0} = \frac{1}{\sqrt{2}} (\ket{0} + \ket{1}) \\
    H\ket{1} = \frac{1}{\sqrt{2}} (\ket{0} - \ket{1})
    ```
    （「量子状態にゲートを作用させる」ことをケットの記法で書くときは、ゲートに対応する記号をケットに左からかけます。）<br/>
    例えば状態$\ket{\psi} = \alpha\ket{0} + \beta\ket{1}$に対しては、
    ```{math}
    \begin{align}
    H\ket{\psi} & = \alpha \frac{1}{\sqrt{2}} (\ket{0} + \ket{1}) + \beta \frac{1}{\sqrt{2}} (\ket{0} - \ket{1}) \\
                & = \frac{1}{\sqrt{2}} (\alpha + \beta) \ket{0} + \frac{1}{\sqrt{2}} (\alpha - \beta) \ket{1}
    \end{align}
    ```
    となる。
  - `circuit.h(i)`
* - $R_{y}$
  - パラメータ$\theta$を取り、計算基底それぞれに対して、以下の変形をする。
    ```{math}
    R_{y}(\theta)\ket{0} = \cos\frac{\theta}{2}\ket{0} + \sin\frac{\theta}{2}\ket{1} \\
    R_{y}(\theta)\ket{1} = -\sin\frac{\theta}{2}\ket{0} + \cos\frac{\theta}{2}\ket{1}
    ```
  - `circuit.ry(theta, i)`
* - $R_{z}$
  - パラメータ$\phi$を取り、計算基底それぞれに対して、以下の変形をする。
    ```{math}
    R_{z}(\phi)\ket{0} = e^{-i\phi/2}\ket{0} \\
    R_{z}(\phi)\ket{1} = e^{i\phi/2}\ket{1}
  - `circuit.rz(phi, i)`
```

それでは、2量子ビットレジスタの第0ビットに$H, R_y, X$の順にゲートをかけて、最後に測定をする回路をQiskitで書いてみましょう。

In [ ]:
# まずは必要になるpythonモジュールをすべてインポートしておく
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, IBMQ, execute
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram

print('notebook ready')

In [ ]:
circuit = QuantumCircuit(2) # レジスタを介さずビット数を指定して回路を作成することもできます
circuit.h(0) # その場合、ゲートにはregister[0]ではなく直接量子ビットの番号を指定します
circuit.ry(np.pi / 2., 0) #　θ = π/2
circuit.x(0)
circuit.measure_all()

print('This circuit has', circuit.num_qubits, 'qubits and', circuit.size(), 'operations')

最後のプリント文で、ゲートが3つなのにも関わらず "5 operations" と出てくるのは、各量子ビットの測定も一つのオペレーションとして数えられるからです。

量子計算に慣れる目的で、この$H, R_y(\pi/2), X$という順の操作で第0ビットに何が起こるかを辿ってみましょう。初期状態は$\ket{0}$で、ケット記法では操作は左からかけていくので、$X R_y(\pi/2) H \ket{0}$を計算することになります。

$$
\begin{align}
X R_y\left(\frac{\pi}{2}\right) H \ket{0} & = X R_y\left(\frac{\pi}{2}\right) \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) \\
& = \frac{1}{\sqrt{2}} X \left[\left(\cos\left(\frac{\pi}{4}\right)\ket{0} + \sin\left(\frac{\pi}{4}\right)\ket{1}\right) + \left(-\sin\left(\frac{\pi}{4}\right)\ket{0} + \cos\left(\frac{\pi}{4}\right)\ket{1}\right)\right] \\
& = \frac{1}{\sqrt{2}} X \frac{1}{\sqrt{2}} \left[\left(\ket{0} + \ket{1}\right) + \left(-\ket{0} + \ket{1}\right)\right] \\
& = X \ket{1} \\
& = \ket{0}
\end{align}
$$

なので、結局$\ket{0}$状態に戻る操作でした。

#### 2量子ビットの操作

2量子ビットの操作は、量子ビットの超電導素子での実装の都合上、全て「制御ゲート」（controlled gates）という方式で行われます。この方式では、2つのビットのうち片方を制御（control）、もう片方を標的（target）として、制御ビットが1の時だけ標的ビットに何らかの操作がかかります。

例として、任意の1ビットゲート$U$を制御ゲート化した$C^i_j[U]$を考えます。ここで$i$が制御、$j$が標的ビットとします。ケットの添字でビットの番号を表して（reminder: 並べて書くときは右から順に番号を振ります）

$$
\begin{align}
C^1_0[U](\ket{0}_1\ket{0}_0) & = \ket{0}_1\ket{0}_0 \\
C^1_0[U](\ket{0}_1\ket{1}_0) & = \ket{0}_1\ket{1}_0 \\
C^1_0[U](\ket{1}_1\ket{0}_0) & = \ket{1}_1U\ket{0}_0 \\
C^1_0[U](\ket{1}_1\ket{1}_0) & = \ket{1}_1U\ket{1}_0
\end{align}
$$

です。

上で紹介した頻出する1ビットゲート$X, Z, H, R_y, R_z$のうち、$H$以外は制御ゲート化バージョンもよく使われます。特に$C[X]$はCXやCNOTとも呼ばれ、量子計算の基本要素として多様されます。実際、全ての2量子ビット制御ゲートはCNOTと1量子ビットゲートの組み合わせに分解できます。

```{list-table}
:header-rows: 1
* - ゲート名
  - 説明
  - Qiskitコード
* - $C^i_j[X]$, CX, CNOT
  - ビット$i$が1である計算基底において、ビット$j$に$X$を作用させる。
  - `circuit.cx(i, j)`
* - $C^i_j[Z]$
  - ビット$i, j$が1である計算基底の符号を反転させる。
  - `circuit.cz(i, j)`
* - $C^i_j[R_{y}]$
  - パラメータ$\theta$を取り、ビット$i$が1である計算基底において、ビット$j$に$R_y$を作用させる。
  - `circuit.cry(theta, i, j)`
* - $C^i_j[R_{z}]$
  - パラメータ$\phi$を取り、ビット$i$が1である計算基底において、ビット$j$に$R_z$を作用させる。
  - `circuit.crz(phi, i, j)`
```

Qiskitで2ビットレジスタに制御ゲートを用い、計算基底$\ket{0}, \ket{1}, \ket{2}, \ket{3}$の振幅の絶対値自乗が$1:2:3:4$の比になるような状態を作ってみましょう。さらに$C^0_1[Z]$ゲートを使って$\ket{3}$だけ振幅の符号が他と異なるようにします。

In [ ]:
theta1 = 2. * np.arctan(np.sqrt(7. / 3.))
theta2 = 2. * np.arctan(np.sqrt(2.))
theta3 = 2. * np.arctan(np.sqrt(4. / 3))

circuit = QuantumCircuit(2)
circuit.ry(theta1, 1)
circuit.ry(theta2, 0)
circuit.cry(theta3 - theta2, 1, 0) # C[Ry]　1が制御で0が標的
circuit.cz(0, 1) # C[Z] 0が制御で1が標的（実はC[Z]ではどちらが制御でも結果は同じ）
circuit.measure_all()

print('This circuit has', circuit.num_qubits, 'qubits and', circuit.size(), 'operations')

やや複雑ですが、また計算を追ってみましょう。まず角度$\theta_1, \theta_2, \theta_3$は定義から以下の関係を満たします。

$$
\begin{align}
R_y(\theta_1)\ket{0} & = \sqrt{\frac{3}{10}} \ket{0} + \sqrt{\frac{7}{10}} \ket{1} \\
R_y(\theta_2)\ket{0} & = \sqrt{\frac{1}{3}} \ket{0} + \sqrt{\frac{2}{3}} \ket{1} \\
R_y(\theta_3 - \theta_2)R_y(\theta_2)\ket{0} & = R_y(\theta_3)\ket{0} = \sqrt{\frac{3}{7}} \ket{0} + \sqrt{\frac{4}{7}} \ket{1}.
\end{align}
$$

したがって、

$$
\begin{align}
& C^1_0[R_y(\theta_3 - \theta_2)]R_{y1}(\theta_1)R_{y0}(\theta_2)\ket{0}_1\ket{0}_0 \\
= & C^1_0[R_y(\theta_3 - \theta_2)]\left(\sqrt{\frac{3}{10}} \ket{0}_1 + \sqrt{\frac{7}{10}} \ket{1}_1\right) R_y(\theta_2)\ket{0}_0\\
= & \sqrt{\frac{3}{10}} \ket{0}_1 R_y(\theta_2)\ket{0}_0 + \sqrt{\frac{7}{10}} \ket{1}_1 R_y(\theta_3)\ket{0}_0 \\
= & \sqrt{\frac{3}{10}} \ket{0}_1 \left(\sqrt{\frac{1}{3}} \ket{0}_0 + \sqrt{\frac{2}{3}} \ket{1}_0\right) + \sqrt{\frac{7}{10}} \ket{1}_1 \left(\sqrt{\frac{3}{7}} \ket{0}_0 + \sqrt{\frac{4}{7}} \ket{1}_0\right) \\
= & \sqrt{\frac{1}{10}} \ket{00} + \sqrt{\frac{2}{10}} \ket{01} + \sqrt{\frac{3}{10}} \ket{10} + \sqrt{\frac{4}{10}} \ket{11} 
\end{align}
$$

最初の行で、ビット0と1にかかる$R_y$ゲートをそれぞれ$R_{y0}, R_{y1}$と表しました。

最後に$C[Z]$をかけると、$\ket{11}$だけ符号が反転します。

### 回路図の書き方と読み方

量子回路を可視化する方法として、「回路図」の標準的な描き方が決まっています。Qiskitでは`QuantumCircuit`オブジェクトの`draw()`というメソッドを使って自動描画できます。

In [ ]:
circuit.draw('mpl')

　ここで`draw()`の引数`'mpl'`はmatplotlibライブラリを使ってカラーで描くことを指定しています。実行環境によっては対応していないこともあるので、その場合は引数なしの`draw()`を使います。結果は`mpl`の場合に比べて見劣りしますが、内容は同じです。

In [ ]:
circuit.draw()

回路図は左から右に読んでいきます。水平の2本の実線が上からそれぞれ第0、第1量子ビットに対応し、その上にかぶさっている四角がゲート、最後にある矢印が下に伸びている箱が測定を表します。1ビットゲートから伸びている先端の丸い縦線は制御を表します。一番下の二重線は「古典レジスタ」に対応し、測定結果の0/1が記録される部分です。

## CHSH不等式を計算する回路を書く

それではいよいよ本題に入りましょう。CHSH不等式は二体系の4つの観測量の関係式なので、2ビットの回路を4つ用意します。そして、それぞれにおいて「ベル状態」$1/\sqrt{2}(\ket{00} + \ket{11})$を実現します。ベル状態は「どちらの量子ビットについても$\ket{0}$でも$\ket{1}$でもない状態」つまり、全体としては一つの定まった（純粋）状態であるにも関わらず、部分を見ると純粋でない状態です。このような時、**二つの量子ビットはエンタングルしている**といいます。エンタングルメントの存在は量子力学の非常に重要な特徴です。

ベル状態はアダマールゲートとCNOTゲートを組み合わせて作ります。ここでは回路を4つ作るので、ループを回して`circuits`という配列に回路を足していきます。

CHSH不等式の検証用の観測量を作るために、量子ビット1に対し、回路0と2では測定の直前に$R_y(-\pi/4)$、1と3では$R_y(-3\pi/4)$を作用させます。また回路2と3では量子ビット0に$R_y(-\pi/2)$を同じく測定の直前に作用させます。

In [ ]:
circuits = []
for ic in range(4):
    circuit = QuantumCircuit(2, name='circuit{}'.format(ic))
    circuit.h(0)
    circuit.cx(0, 1)
    circuits.append(circuit)

circuits[0].ry(-np.pi / 4., 1)
circuits[1].ry(-3. * np.pi / 4., 1)
circuits[2].ry(-np.pi / 4., 1)
circuits[3].ry(-3. * np.pi / 4., 1)

circuits[2].ry(-np.pi / 2., 0)
circuits[3].ry(-np.pi / 2., 0)

for circuit in circuits:
    circuit.measure_all()

# draw()にmatplotlibのaxesオブジェクトを渡すと、そこに描画してくれる
# 一つのノートブックセルで複数プロットしたい時などに便利
for circuit in circuits:
    ax = plt.figure().add_subplot()
    circuit.draw('mpl', ax=ax)

それぞれの回路で2ビットレジスタの基底$\ket{00}, \ket{01}, \ket{10}, \ket{11}$が現れる確率を計算してみましょう。

回路0の状態は

$$
\begin{align}
R_{y1}\left(-\frac{\pi}{4}\right) C^0_1[X] H_0 \ket{0}_1\ket{0}_0 = & R_{y1}\left(-\frac{\pi}{4}\right) \frac{1}{\sqrt{2}} (\ket{0}_1\ket{0}_0 + \ket{1}_1\ket{1}_0) \\
= & \frac{1}{\sqrt{2}} \left[(c\ket{0}_1 - s\ket{1}_1)\ket{0}_0 + (s\ket{0}_1 + c\ket{1}_1)\ket{1}_0\right]\\
= & \frac{1}{\sqrt{2}} (c\ket{00} + s\ket{01} - s\ket{10} + c\ket{11}).
\end{align}
$$

簡単のため$c = \cos(\pi/8), s = \sin(\pi/8)$とおきました。

したがって回路0での確率$P^0_{l} \, (l=00,01,10,11)$は

$$
P^0_{00} = P^0_{11} = \frac{c^2}{2} \\
P^0_{01} = P^0_{10} = \frac{s^2}{2}
$$

同様に、回路1の状態は

$$
R_{y1}\left(-\frac{3\pi}{4}\right) \frac{1}{\sqrt{2}} (\ket{0}_1\ket{0}_0 + \ket{1}_1\ket{1}_0) = \frac{1}{\sqrt{2}} (s\ket{00} + c\ket{01} - c\ket{10} + s\ket{11})
$$

で確率$P^1_{l}$は

$$
P^1_{00} = P^1_{11} = \frac{s^2}{2} \\
P^1_{01} = P^1_{10} = \frac{c^2}{2}
$$

です。回路2の状態は

$$
\begin{align}
& R_{y1}\left(-\frac{\pi}{4}\right) R_{y0}\left(-\frac{\pi}{2}\right) \frac{1}{\sqrt{2}} (\ket{0}_1\ket{0}_0 + \ket{1}_1\ket{1}_0) \\
= & \frac{1}{\sqrt{2}} \left[ \frac{1}{\sqrt{2}} (c\ket{0}_1 - s\ket{1}_1) (\ket{0}_0 - \ket{1}_0) + \frac{1}{\sqrt{2}} (s\ket{0}_1 + c\ket{1}_1) (\ket{0}_0 + \ket{1}_0) \right] \\
= & \frac{1}{2} \left[ (s+c)\ket{00} + (s-c)\ket{01} - (s-c)\ket{10} + (s+c)\ket{11} \right]
\end{align}
$$

で確率$P^2_{l}$は

$$
P^2_{00} = P^2_{11} = \frac{(s + c)^2}{4} \\
P^2_{01} = P^2_{10} = \frac{(s - c)^2}{4}
$$

同様に回路3の状態と確率$P^3_i$は

$$
\begin{align}
& R_{y1}\left(-\frac{3\pi}{4}\right) R_{y0}\left(-\frac{\pi}{2}\right) \frac{1}{\sqrt{2}} (\ket{0}_1\ket{0}_0 + \ket{1}_1\ket{1}_0) \\
= & \frac{1}{2} \left[ (s+c)\ket{00} - (s-c)\ket{01} + (s-c)\ket{10} + (s+c)\ket{11} \right]
\end{align}
$$

$$
P^3_{00} = P^3_{11} = \frac{(s + c)^2}{4} \\
P^3_{01} = P^3_{10} = \frac{(s - c)^2}{4}
$$

となります。

それぞれの回路でビット0と1で同じ値が観測される確率$P^{i}_{00} + P^{i}_{11}$から異なる値が観測される確率$P^{i}_{01} + P^{i}_{10}$を引いた値を$C^{i}$と定義します。

$$
C^0 = c^2 - s^2 = \cos\left(\frac{\pi}{4}\right) = \frac{1}{\sqrt{2}} \\
C^1 = s^2 - c^2 = -\frac{1}{\sqrt{2}} \\
C^2 = 2sc = \sin\left(\frac{\pi}{4}\right) = \frac{1}{\sqrt{2}} \\
C^3 = 2sc = \frac{1}{\sqrt{2}}
$$

なので、これらの組み合わせ$S = C^0 - C^1 + C^2 + C^3$の値は$2\sqrt{2}$です。

実は、エンタングルメントが起こらない場合、この観測量$S$の値は2を超えられないことが知られています。例えば$R_y$ゲートをかける前の状態がベル状態ではなく、確率$\frac{1}{2}$で$\ket{00}$、確率$\frac{1}{2}$で$\ket{11}$という「混合状態」である場合、

$$
C^0 = \frac{1}{\sqrt{2}} \\
C^1 = -\frac{1}{\sqrt{2}} \\
C^2 = 0 \\
C^3 = 0
$$

となり、$S = \sqrt{2} < 2$です。これがCHSH不等式です。

それでは、IBMQの「量子コンピュータ」が実際にエンタングル状態を生成できるのか、上の四つの回路から$S$の値を計算して確認してみましょう。

## 回路を実機で実行する

まずは{ref}`トークン <qiskit_token>`を使ってIBMQに接続します。

In [ ]:
IBMQ.enable_account('__paste_your_token_here__')

認証が済んだら、利用する量子コンピュータ（「バックエンド」と呼びます）を選びます。

In [ ]:
# IBMQプロバイダ（実機へのアクセスを管理するオブジェクト）
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# バックエンド（実機）のうち量子ビット数2個以上のもののリストをプロバイダから取得し、一番空いているものを選ぶ
backend_filter = lambda b: (not b.configuration().simulator) and (b.configuration().n_qubits >= 2) and b.status().operational
backend = least_busy(provider.backends(filters=backend_filter))

print('Jobs will run on', backend.name())

回路をバックエンドに送るには、`execute`関数を使います。引数`shots`で回路の実行と測定を何回繰り返すか指定します。バックエンドごとに一度のジョブでの最大ショット数が決められていますが、大半の場合8192 (=2<sup>13</sup>)回となっています。

一つのジョブで複数の回路を`shots`回ずつ実行できます。一度に送れる回路の数はバックエンドごとに決まっています。

In [ ]:
shots = 8192

job = execute(circuits, backend=backend, shots=shots)

job_monitor(job, interval=2)

これで回路がジョブとしてバックエンドに送られ、キューに入りました。ジョブの実行結果は`execute`関数の返り値であるジョブオブジェクトから参照します。

IBMQのバックエンドは世界中からたくさんのユーザーに利用されているため、場合によっては予約されているジョブが多数あってキューにかなりの待ち時間が生じることがあります。

バックエンドごとのキューの長さは[IBM Quantum Experience](https://quantum-computing.ibm.com/)のホーム画面の右側で確認できます。この右側のコラムでバックエンドを一つクリックすると詳細が表示され、一番下の Your access providers という欄でバックエンドの最大ショット数と最大ジョブ数を確認できます。

また、自分の投じたジョブのステータスは左のツールバーのうちのJobsボタンから確認できます。

Qiskitプログラム中からもジョブのステータスを確認できます。いくつか方法がありますが、シンプルに一つのジョブをテキストベースでモニターするだけなら上のように`job_monitor`を使います。

## 量子測定結果の解析

ジョブオブジェクトの`result()`というメソッドを呼ぶと、ジョブが完了して結果が帰ってくるまでコードの実行が止まります。実行結果はオブジェクトとして返され、それの`get_counts`というメソッドを使うと、各ビット列が何回観測されたかというヒストグラムデータがPythonのdictとして得られます。

In [ ]:
result = job.result()

counts = []
for circuit in circuits:
    c = result.get_counts(circuit)
    counts.append(c)
    
print(counts)

In [ ]:
# !! Dummy contents -- delete this cell
counts = [
    {'00': 3339, '01': 720, '10': 863, '11': 3270},
    {'00': 964, '01': 3332, '10': 3284, '11': 612},
    {'00': 3414, '01': 693, '10': 953, '11': 3132},
    {'00': 3661, '01': 725, '10': 768, '11': 3038}
]

````{tip}
ノートブックの接続が切れてしまったり、過去に走らせたジョブの結果を再び解析したくなったりした場合は、ジョブIDを使って`retrieve_job`というメソッドでジョブオブジェクトを再構成することができます。過去に走らせたジョブはIBM Quantum Experienceのホームページにリストされているので、そこにあるジョブID（603d8bef43659838aのような）をコピーし、

```{code-block} python
backend = provider.get_backend('backend_you_used')
job = backend.retrieve_job('job_id')
```

とすると、`execute`によって返されたのと同じようにジョブオブジェクトが生成されます。
````

Qiskitから提供されている`plot_histogram`関数を使って、この情報を可視化できます。プロットの縦軸は観測回数を全測定数で割って、観測確率に規格化してあります。

In [ ]:
for c in counts:
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)

上で、$c^2 = (\sqrt{2} + 1) / 2\sqrt{2} \sim 0.85$, $s^2 = (\sqrt{2} - 1) / 2\sqrt{2} \sim 0.15$なので、得られた確率は当たらずとも遠からずというところでしょうか。

実は現在の量子コンピュータにはまだ様々なノイズやエラーがあり、計算結果は往々にして理論的な値から統計誤差の範囲を超えてずれます。特定のエラーに関しては多少の緩和法も存在しますが、全て防げるわけでは決してありません。現在の量子コンピュータを指して "*Noisy* intermediate-scale quantum (NISQ) device" と呼んだりしますが、このNoisyの部分はこのような簡単な実験でもすでに顕著に現れるわけです。

逆に、NISQデバイスを有効活用するには、ノイズやエラーがあっても意味のある結果が得られるようなロバストな回路が求められます。{doc}`vqe`で紹介する変分量子回路を用いた最適化などがその候補として注目されています。

さて、それでは最後にCHSH不等式の破れを確認してみましょう。$C^{0}, C^{1}, C^{2}, C^{3}$を計算して$S$を求めます。

In [ ]:
C = []
for c in counts:
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]

print('C:', C)
print('S =', S)
if S > 2.:
    print('Yes, we are using a quantum computer!')

無事、$S$が2を超えました。

## 参考文献

```{bibliography}
:filter: docname in docnames
```